<a href="https://colab.research.google.com/github/OlegPozovnoy/MailRuMLHomeworks/blob/master/HW3/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/datasets/')

Drive already mounted at /datasets/; to attempt to forcibly remount, call drive.mount("/datasets/", force_remount=True).


In [2]:
!ls "/datasets/My Drive/datasets/"

players.pkl  results.pkl  tournaments.pkl


In [0]:
import pickle
import pandas as pd

In [0]:
tournaments = pd.read_pickle("/datasets/My Drive/datasets/tournaments.pkl")

tournaments_cleaned = [(v['id'], v['name'], v['dateStart'][0:4], v['type']['name'], v['questionQty']) for k,v in tournaments.items() if v['questionQty'] != None]
del tournaments

Создадим трейн и тест набор турниров.

In [5]:
train_id = [ v[0] for v in tournaments_cleaned if int(v[2])<= 2019]
test_id = [v[0] for v in tournaments_cleaned if int(v[2])== 2020]

print(len(train_id))
print(len(test_id))

3921
418


In [0]:
results = pd.read_pickle("/datasets/My Drive/datasets/results.pkl")

results_cleaned = [(k, v) for k,v in results.items() if k in train_id]

results_cleaned2 = []
for t in results_cleaned:
  for v in t[1]:
    results_cleaned2.append((t[0],v.get('mask'),v.get('questionsTotal'), [s.get('player').get('id') for s in v.get('teamMembers')]))

del results
del results_cleaned



In [9]:
import numpy as np
from sklearn.linear_model import LogisticRegression

print (len(results_cleaned2))
vocabulary =  set()


def process_tournament(id):
  task3 = []
  words = []
  for v in results_cleaned2:
    if v[1] != None and v[0] == id :
      for p in range(len(v[1])):
        for k in v[3]:
          quest = str(v[0])+'_'+str(p)
          player = int(k)
          task3.append((quest, player, v[1][p]) )
          words.append(quest)
          words.append(player)

  if len(words)==0:
    return []
  
  words = set(words)
  word_to_idx = dict([(y,x) for x,y in enumerate(words)])
  idx_to_word = dict([(x,y) for x,y in enumerate(words)])

  
  X = np.zeros((len(task3),len(words)))
  Y = np.zeros(len(task3))
  for i in range(len(task3)):
    X[i,word_to_idx[task3[i][0]]] = 1
    X[i,word_to_idx[task3[i][1]]] = 1
    Y[i] = task3[i][2]

  logistic = LogisticRegression(fit_intercept = False ) #penalty = 'none', 
  logistic.fit(X, Y)
  #logistic.score(X, Y)

  result = []
  for s in range(len(logistic.coef_[0])):
    result.append((idx_to_word[s],logistic.coef_[0][s]))
  #print('R² Value: \n', logistic.score(X, Y))
  return result

print(process_tournament(1521))
process_tournament(21)

436367
[(32777, 1.4151233958565705), (8206, 0.4029578415112407), (32789, 0.9221965288792394), (31, -0.470110346943416), (40998, 1.536319759993968), (16429, -0.3136126595532352), (47, 1.0471974039349707), (8252, -2.042325352206522), (16454, -0.9781158911836383), (16471, -0.8013938321940022), (32857, -0.015332333582343089), (16478, -0.8013938321940026), (32870, -0.3136126595532352), (16488, -0.9781158911836381), (24689, 1.293387609572403), (24690, 0.6670016820358808), (119, -1.3606138490856705), (32892, -0.3136126595532352), (8317, -0.3136126595532352), (24716, -0.4701103469434161), (8352, 0.9221965288792395), (16549, 0.5362361667918285), (16565, -0.1622165977508994), (32960, -1.796332346781391), (196, 0.402957841511241), (16581, 0.6670016820358806), (32967, -0.6324163939827953), ('1521_42', -1.2485134879155615), (41166, 0.5362361667918283), (41167, 0.5362361667918283), (41170, 0.6670016820358803), (32979, 1.536319759993968), (223, 1.0471974039349705), (24813, 1.0471974039349707), (16621

[]

Создадим словарь из всех вопросов и игроков, их можно отличать по наличию символа '_' в названии.

In [0]:
all_words = []

for v in results_cleaned2:
  if v[1] != None:
    for p in range(len(v[1])):
      for k in v[3]:
        all_words.append(str(v[0])+'_'+str(p))
        all_words.append(k)

all_words =  set(all_words)


Пройдемся по всем турнирам и посчитаем рейтинг всех игроков и вопросов в заданном турнире, за неимением лучшего, усредним по каждому игроку рейтинг в турнире

In [0]:
import tqdm
import gc
all_results = {}
for s in all_words:
  all_results[s] = [0,0]

print(all_results)

i=0
for tourn in tqdm.tqdm(train_id):
  i=i+1
  print(tourn)
  result = process_tournament(tourn)
  for s in result:
    all_results[s[0]][1] +=1
    all_results[s[0]][0] +=s[1]
  gc.collect()

  0%|          | 0/3921 [00:00<?, ?it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

  0%|          | 1/3921 [00:00<39:42,  1.65it/s]

76


  0%|          | 2/3921 [00:01<47:48,  1.37it/s]

141


  0%|          | 3/3921 [00:02<50:09,  1.30it/s]

...

1519


  1%|          | 24/3921 [00:30<3:41:31,  3.41s/it]

1521


  1%|          | 25/3921 [00:38<5:11:42,  4.80s/it]

1526


In [14]:
#print(results_cleaned2[314])
#print(results_cleaned2[1518])

(314, None, 0, [])


In [51]:
ranking = [(k,sum(v)/len(v),len(v)) for k,v in all_results.items() if len(v)>0]
print(ranking)

ranking.sort(key = lambda x: -x[1])

print(len(ranking))

[(144, 0.8115716822453486, 3921), (439, 0.8115716822453486, 3921), (606, 0.3096916142463704, 3921), (668, 1.2453003007722203, 3921), (707, 1.81281595304428, 3921), ('22_61', -0.8014541582410943, 3921), (1055, -0.024618723036230465, 3921), (1145, 0.8115716822453486, 3921), (1560, 2.021934955245346, 3921), (1603, 1.3353444131489387, 3921), (2005, 1.1565697901447998, 3921), (2454, 1.1565697901447998, 3921), (2626, 0.3096916142463704, 3921), (2694, 1.81281595304428, 3921), ('22_89', 2.3396746890502476, 3921), (2935, 2.021934955245346, 3921), (3207, 1.5201563477227515, 3921), (3270, 2.021934955245346, 3921), (3498, -1.2127987362172403, 3921), (3645, 1.5201563477227515, 3921), (4060, 1.2453003007722203, 3921), (4551, 1.1565697901447998, 3921), (4576, 0.9823792319043849, 3921), ('22_46', -20.77005188280133, 3921), (4878, 2.021934955245346, 3921), (5526, 1.7128253338578534, 3921), (5935, 1.3353444131489387, 3921), (6020, 0.4761072327512964, 3921), (6212, 1.5201563477227515, 3921), (6625, 0.811

In [0]:
def isquestion():
  if 

24